<h1 align="center"> Web Scrapy Project - Crypto Punk! </h1>
<h2 align="center"> <font size='4'>Chenchen Liu </h2>

This web scrapy project extracts data from crypto punk website where pixel art images are traded as NFT on Ethereum. Using various punk data including index, type, attribute, ownership, price, transaction frequency, transaction type and transaction date to analyze and predict its market value. Please note 10,000 punks are collected from web scrapy and the project is only for education purpose.

## 1. Data Collection

In [4]:
from bs4 import BeautifulSoup # For HTML parsing
import requests # Website connections
from time import sleep # To prevent overwhelming the server between connections
import pandas as pd # For converting results to a dataframe and bar chart plots
import numpy as np # For data cleaning
import random # To generater random number


- Get webpage information from cryptopunk website

In [807]:
def get_page(page_num):
    url = f'https://www.larvalabs.com/cryptopunks/details/{page_num}'
    result = requests.get(url)
    soup = BeautifulSoup(result.content)
    return soup

In [903]:
#Get attribute of cryptopunk, save data to csv file

index = []
number = []

for page_num in range(0 ,10000):
    soup = get_page(page_num)

    sleep(2) 
        
    for tag in soup.find_all('div', class_='col-md-4')[1:]:
        try:
            number.append(tag.a.get_text())
        except:
            number.append('error')
        index.append(page_num)
        mapped = list(zip(index, number))
        
        if page_num % 1111 == 0:
            pd.DataFrame(mapped, columns=['number', 'attribute']).to_csv(f'attribute_data-{page_num}.csv')


In [906]:
#cryptopunk main dataframe, save data to csv file

data = []

for page_num in range(0,10000):
    soup = get_page(page_num)
    
    sleep(6) 
    
    # Get the index number of cryptopunk
    try:
        number = soup.find('li',class_='active').text
    except:
        number = 'error'
        
    # Get the character type of cryptopunk   
    try:
        type_ = soup.find('div',class_='col-md-10 col-md-offset-1 col-xs-12').a.text
    except:
        type_ = 'error'
    
    # Get the initial owner of cryptopunk
    try:
        initial_owner = soup.find("tr",{"class":"punk-history-row-claim"}).a.text
    except:
        initial_owner = 'error'
    
    # Get the current owner of cryptopunk
    try:
        current_owner = soup.find_all("div",{"class":"col-md-10 col-md-offset-1"})[1].a.text
    except:
        current_owner = 'error'
    
    # Get the latest transaction type of cryptopunk
    try:
        type_TX = soup.find("table",{"class":"table"}).find_all("tr")[1].find_all("td")[0].text
    except:
        type_TX = 'error'
    
    # Get the initial_market_value of cryptopunk
    try:
        initial_market_value = soup.find("table",{"class":"table"}).find_all("tr")[-2].find_all("td")[3].text
    except:
        initial_market_value = 0
        
    # Get the latest_market_value of cryptopunk 
    try:
        latest_market_value = soup.find("table",{"class":"table"}).find_all("tr")[1].find_all("td")[3].text
    except:
        latest_market_value = 'error'
    
    # Get total number of transactions of cryptopunk
    try:
        transaction_frequency = pd.read_html(requests.get(f'https://www.larvalabs.com/cryptopunks/details/{page_num}').text)[0].shape[0]-1
    except:
        transaction_frequency = 'error'
    
   # Get the initial transaction date of cryptopunk
    try:
        initial_TXD = soup.find("table",{"class":"table"}).find_all("tr")[-2].find_all("td")[4].text
    except:
        initial_TXD = 'error'
        
    # Get the latest transaction date of cryptopunk: 
    try:
        latest_TXD = soup.find("table",{"class":"table"}).find_all("tr")[1].find_all("td")[4].a.text
    except:
        latest_TXD = 'error'
    
    # Combine all the attributes into dictionary
    row = {'number': number, 
           'punk_type': type_, 
           'initial_owner': initial_owner,
           'current_owner': current_owner,
           'transaction_type': type_TX,
           'transaction_frequency': transaction_frequency,
           'initial_market_value': initial_market_value,
           'latest_market_value': latest_market_value,
           'initial_TXD': initial_TXD,
           'latest_TXD' :latest_TXD
          }
    
    data.append(row)
    
    # Save to csv file
    if page_num % 1111 == 0:
        pd.DataFrame(data).to_csv(f'pd_data-{page_num}.csv')

    

## 2. Data Cleaning

1. Clean crypto punk attribute data

In [5]:
df1 = pd.read_csv("attribute_data.csv")
df1

,Unnamed: 0,number,attribute
0,0,0,Green Eye Shadow
1,1,0,Earring
2,2,0,Blonde Bob
3,3,1,Smile
4,4,1,Mohawk
...,...,...,...
21260,21260,9998,Wild White Hair
21261,21261,9998,Black Lipstick
21262,21262,9998,Clown Eyes Green
21263,21263,9999,Mohawk


In [6]:
# remove column 'Unnamed: 0' in df1
df1 = df1.drop(columns=['Unnamed: 0'])

In [7]:
df1

,number,attribute
0,0,Green Eye Shadow
1,0,Earring
2,0,Blonde Bob
3,1,Smile
4,1,Mohawk
...,...,...
21260,9998,Wild White Hair
21261,9998,Black Lipstick
21262,9998,Clown Eyes Green
21263,9999,Mohawk


In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21265 entries, 0 to 21264
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   number     21265 non-null  int64 
 1   attribute  21265 non-null  object
dtypes: int64(1), object(1)
memory usage: 332.4+ KB


2. Clean crypto punk main dataframe

In [9]:
df2 = pd.read_csv('punk_data_UTF8.csv')

In [10]:
df2.head(20)

,Unnamed: 0,number,punk_type,initial_owner,current_owner,transaction_type,transaction_frequency,initial_market_value,latest_market_value,initial_TXD,latest_TXD
0,0,0,Female,0xc352b5,0xe08c32,Bid Withdrawn,34,0.67?? ($219),250?? ($1.03M),23-Jun-17,10-May-21
1,1,1,Male,0xc352b5,0xb88f61,Transfer,52,0.20?? ($61),NaN,24-Jun-17,30-Nov-20
2,2,2,Female,0xc352b5,0x897aea,Transfer,25,0.42?? ($117),NaN,25-Jun-17,09-Jun-21
3,3,3,Male,0xc352b5,0xc352b5,Bid Withdrawn,14,0.05?? ($13),"33?? ($11,638)",26-Jun-17,04-Oct-20
4,4,4,Male,0xc352b5,0xc352b5,Bid Withdrawn,11,0.10?? ($28),"28?? ($109,026)",25-Jun-17,13-May-21
5,5,5,Female,0xc352b5,WrappedC?€?,Bid,13,0.06?? ($15),"4.20?? ($11,322)",26-Jun-17,29-Apr-21
6,6,6,Male,0xc352b5,0xc352b5,Bid Withdrawn,6,0.06?? ($15),"10?? ($10,126)",26-Jun-17,04-Jan-21
7,7,7,Female,0xc352b5,0xc352b5,Bid Withdrawn,11,0.06?? ($15),1.75?? ($551),26-Jun-17,26-Jul-20
8,8,8,Male,0xc352b5,0xc352b5,Bid Withdrawn,18,0.06?? ($15),"30?? ($68,985)",26-Jun-17,13-Apr-21
9,9,9,Male,0xc352b5,0xc352b5,Bid Withdrawn,30,0.06?? ($15),"36?? ($13,597)",26-Jun-17,14-Oct-20


In [11]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Unnamed: 0             10000 non-null  int64 
 1   number                 10000 non-null  int64 
 2   punk_type              10000 non-null  object
 3   initial_owner          10000 non-null  object
 4   current_owner          10000 non-null  object
 5   transaction_type       10000 non-null  object
 6   transaction_frequency  10000 non-null  int64 
 7   initial_market_value   7683 non-null   object
 8   latest_market_value    5541 non-null   object
 9   initial_TXD            10000 non-null  object
 10  latest_TXD             10000 non-null  object
dtypes: int64(3), object(8)
memory usage: 859.5+ KB


In [12]:
# remove column 'Unnamed: 0' in df1
df2 = df2.drop(columns=['Unnamed: 0'])

In [13]:
# convert initial_TXD and latest_TXD to datetime data type
df2['initial_TXD'] =  pd.to_datetime(df2['initial_TXD'], format = '%d-%b-%y')
df2['latest_TXD'] =  pd.to_datetime(df2['latest_TXD'], format = '%d-%b-%y')

In [14]:
# Add year of initial_TXD and latest_TXD in tow new columns
df2['initial_TXY'] = df2['initial_TXD'].dt.year
df2['latest_TXY'] = df2['latest_TXD'].dt.year

In [15]:
#convert initial_market_value to US dollar

df2['initial_market_value'] = df2['initial_market_value'].str.replace('$', '').str.findall('\((.*)\)').str.get(0) \
                                                         .str.replace(',','').str.replace('<','').str.replace('Y','').fillna(0) 

df2['initial_market_value'] = (df2['initial_market_value'].replace(r'[ZMP]+$', '', regex=True).astype(float) * \
                               df2['initial_market_value'].str.extract(r'[\d\.]+([ZMP]+)', expand=False)
                               .fillna(1)
                               .replace(['Z','M', 'P'], [10**15, 10**6, 10**15]).astype(float))

#convert latest_market_value to US dollar

df2['latest_market_value'] = df2['latest_market_value'].str.replace('$', '').str.findall('\((.*)\)').str.get(0) \
                                                       .str.replace(',','').str.replace('<','').str.replace('Y','').fillna(0) 

df2['latest_market_value']= (df2['latest_market_value'].replace(r'[ZMB]+$', '', regex=True).astype(float) * \
                             df2['latest_market_value'].str.extract(r'[\d\.]+([ZMB]+)', expand=False)
                             .fillna(1)
                             .replace(['Z','M', 'B'], [10**15, 10**6, 10**9]).astype(float))                                                       



In [16]:
#convert transaction_frequency to int
df2['transaction_frequency'] = pd.to_numeric(df2['transaction_frequency'], errors='coerce').convert_dtypes() 

In [17]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   number                 10000 non-null  int64         
 1   punk_type              10000 non-null  object        
 2   initial_owner          10000 non-null  object        
 3   current_owner          10000 non-null  object        
 4   transaction_type       10000 non-null  object        
 5   transaction_frequency  10000 non-null  Int64         
 6   initial_market_value   10000 non-null  float64       
 7   latest_market_value    10000 non-null  float64       
 8   initial_TXD            10000 non-null  datetime64[ns]
 9   latest_TXD             10000 non-null  datetime64[ns]
 10  initial_TXY            10000 non-null  int64         
 11  latest_TXY             10000 non-null  int64         
dtypes: Int64(1), datetime64[ns](2), float64(2), int64(3), object(

In [18]:
df2

,number,punk_type,initial_owner,current_owner,transaction_type,transaction_frequency,initial_market_value,latest_market_value,initial_TXD,latest_TXD,initial_TXY,latest_TXY
0,0,Female,0xc352b5,0xe08c32,Bid Withdrawn,34,219.0,1030000.0,2017-06-23,2021-05-10,2017,2021
1,1,Male,0xc352b5,0xb88f61,Transfer,52,61.0,0.0,2017-06-24,2020-11-30,2017,2020
2,2,Female,0xc352b5,0x897aea,Transfer,25,117.0,0.0,2017-06-25,2021-06-09,2017,2021
3,3,Male,0xc352b5,0xc352b5,Bid Withdrawn,14,13.0,11638.0,2017-06-26,2020-10-04,2017,2020
4,4,Male,0xc352b5,0xc352b5,Bid Withdrawn,11,28.0,109026.0,2017-06-25,2021-05-13,2017,2021
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,Female,Punkofth?€?,0x758353,Bid Withdrawn,6,16.0,63960.0,2017-06-28,2021-04-20,2017,2021
9996,9996,Male,0xa9bfa9,0xa9bfa9,Bid Withdrawn,4,11.0,1338.0,2017-08-04,2021-01-20,2017,2021
9997,9997,Zombie,TJ2010,0x279679,Offered,36,46.0,1480000.0,2017-06-24,2021-05-17,2017,2021
9998,9998,Female,cyberpun?€?,0xcfd9cc,Offer Withdrawn,9,4292.0,0.0,2017-06-27,2021-05-04,2017,2021


In [19]:
#check extreme value in initial_market_value
df2[df2['initial_market_value']>10000000]

,number,punk_type,initial_owner,current_owner,transaction_type,transaction_frequency,initial_market_value,latest_market_value,initial_TXD,latest_TXD,initial_TXY,latest_TXY
4342,4342,Male,0x2bcbbd,Tschuuul?€?,Sold,5,3.137773e+08,9.100000e+01,2017-11-03,2019-04-08,2017,2019
4714,4714,Male,tonyherr?€?,tonyherr?€?,Offered,1,3.927814e+38,3.927814e+38,2021-05-03,2021-05-03,2021,2021
4722,4722,Female,tonyherr?€?,tonyherr?€?,Offered,1,3.377685e+38,3.377685e+38,2021-05-01,2021-05-01,2021,2021
4727,4727,Male,tonyherr?€?,tonyherr?€?,Offered,1,2.875873e+38,2.875873e+38,2021-06-08,2021-06-08,2021,2021
6141,6141,Female,0x58164b,0x717403,Bid,4,2.741000e+16,6.400000e+01,2017-06-24,2020-12-18,2017,2020
6748,6748,Male,0x0873fc,0x6abc0b,Bid Withdrawn,47,2.688000e+17,6.072000e+03,2017-07-06,2021-06-13,2017,2021


In [20]:
#check extreme value in latest_market_value
df2[df2['latest_market_value']>10000000]

,number,punk_type,initial_owner,current_owner,transaction_type,transaction_frequency,initial_market_value,latest_market_value,initial_TXD,latest_TXD,initial_TXY,latest_TXY
547,547,Female,0xc352b5,Tschuuul?€?,Offered,6,0.000000e+00,1.156000e+07,2021-02-18,2021-02-22,2021,2021
950,950,Male,0xc352b5,Tschuuul?€?,Offered,24,0.000000e+00,2.177700e+08,2019-02-08,2021-02-22,2019,2021
1357,1357,Female,Punkofth?€?,0x758353,Offered,2,0.000000e+00,2.840418e+38,2021-02-06,2021-04-16,2021,2021
1477,1477,Female,shilpixe?€?,Tschuuul?€?,Offered,3,1.190000e+02,3.805800e+20,2019-05-16,2020-10-13,2019,2020
1750,1750,Female,0x00bd9f,0x671f71,Offered,20,1.300000e+01,1.681500e+08,2017-06-26,2021-03-22,2017,2021
2106,2106,Male,0x00d7c9,0x592234,Offered,17,1.260000e+02,1.283000e+07,2018-01-12,2021-03-20,2018,2021
2204,2204,Female,0xcbd482,0x577ebc,Offered,78,5.100000e+01,4.015504e+38,2017-06-26,2021-05-04,2017,2021
2294,2294,Male,0x365e35,0xff0bd4,Offered,19,1.600000e+01,1.559090e+27,2017-06-28,2021-03-02,2017,2021
2345,2345,Male,Punkofth?€?,0x758353,Offered,2,0.000000e+00,4.424129e+38,2021-02-06,2021-05-08,2021,2021
2468,2468,Male,Punkofth?€?,0x758353,Offered,2,0.000000e+00,3.071884e+38,2021-02-06,2021-06-04,2021,2021


3. Combine df1 amd df2 together

In [21]:
df = pd.merge(df1, df2, left_on='number', right_on='number', how='left')
df.head(20)

,number,attribute,punk_type,initial_owner,current_owner,transaction_type,transaction_frequency,initial_market_value,latest_market_value,initial_TXD,latest_TXD,initial_TXY,latest_TXY
0,0,Green Eye Shadow,Female,0xc352b5,0xe08c32,Bid Withdrawn,34,219.0,1030000.0,2017-06-23,2021-05-10,2017,2021
1,0,Earring,Female,0xc352b5,0xe08c32,Bid Withdrawn,34,219.0,1030000.0,2017-06-23,2021-05-10,2017,2021
2,0,Blonde Bob,Female,0xc352b5,0xe08c32,Bid Withdrawn,34,219.0,1030000.0,2017-06-23,2021-05-10,2017,2021
3,1,Smile,Male,0xc352b5,0xb88f61,Transfer,52,61.0,0.0,2017-06-24,2020-11-30,2017,2020
4,1,Mohawk,Male,0xc352b5,0xb88f61,Transfer,52,61.0,0.0,2017-06-24,2020-11-30,2017,2020
5,2,Wild Hair,Female,0xc352b5,0x897aea,Transfer,25,117.0,0.0,2017-06-25,2021-06-09,2017,2021
6,3,Wild Hair,Male,0xc352b5,0xc352b5,Bid Withdrawn,14,13.0,11638.0,2017-06-26,2020-10-04,2017,2020
7,3,Nerd Glasses,Male,0xc352b5,0xc352b5,Bid Withdrawn,14,13.0,11638.0,2017-06-26,2020-10-04,2017,2020
8,3,Pipe,Male,0xc352b5,0xc352b5,Bid Withdrawn,14,13.0,11638.0,2017-06-26,2020-10-04,2017,2020
9,4,Big Shades,Male,0xc352b5,0xc352b5,Bid Withdrawn,11,28.0,109026.0,2017-06-25,2021-05-13,2017,2021


## 3. Data Visualization

In [22]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.offline as py
import plotly.graph_objs as go 
import plotly.express as px

py.init_notebook_mode(connected=True)

- Estimated average annual investment return rate

In [23]:
#filter valid initial market value, latest market value adn transaction year

df_irr = df2[(df2['initial_market_value'] > 1) & (df2['initial_market_value'] < 10000000) & \
             (df2['latest_market_value'] > 1) & (df2['latest_market_value'] < 10000000) & \
             (df2['initial_TXY'] != df2['latest_TXY'])]
df_irr

,number,punk_type,initial_owner,current_owner,transaction_type,transaction_frequency,initial_market_value,latest_market_value,initial_TXD,latest_TXD,initial_TXY,latest_TXY
0,0,Female,0xc352b5,0xe08c32,Bid Withdrawn,34,219.0,1030000.0,2017-06-23,2021-05-10,2017,2021
3,3,Male,0xc352b5,0xc352b5,Bid Withdrawn,14,13.0,11638.0,2017-06-26,2020-10-04,2017,2020
4,4,Male,0xc352b5,0xc352b5,Bid Withdrawn,11,28.0,109026.0,2017-06-25,2021-05-13,2017,2021
5,5,Female,0xc352b5,WrappedC?€?,Bid,13,15.0,11322.0,2017-06-26,2021-04-29,2017,2021
6,6,Male,0xc352b5,0xc352b5,Bid Withdrawn,6,15.0,10126.0,2017-06-26,2021-01-04,2017,2021
...,...,...,...,...,...,...,...,...,...,...,...,...
9992,9992,Male,0xba7ccc,0xcffc33,Bid Withdrawn,5,16.0,82439.0,2017-06-28,2021-05-05,2017,2021
9995,9995,Female,Punkofth?€?,0x758353,Bid Withdrawn,6,16.0,63960.0,2017-06-28,2021-04-20,2017,2021
9996,9996,Male,0xa9bfa9,0xa9bfa9,Bid Withdrawn,4,11.0,1338.0,2017-08-04,2021-01-20,2017,2021
9997,9997,Zombie,TJ2010,0x279679,Offered,36,46.0,1480000.0,2017-06-24,2021-05-17,2017,2021


In [24]:
IRR = (df_irr['latest_market_value']/df_irr['initial_market_value'])**(1/(df_irr['latest_TXY']-df_irr['initial_TXY']))
IRR.agg('mean')

99.60523193547984

- Quantity distribution of punk type \
  98% of punk types are male and female, rare punk tyles are zombie, Ape and Alien.

In [25]:
punk_types = df2['punk_type'].value_counts()

data1 = go.Bar(x=punk_types.values, y=punk_types.index, orientation='h', text=punk_types.values, textposition='auto')

layout1 = go.Layout(title='Crypto punk type counts',
                    xaxis={'title': 'Quantity'},
                    yaxis={'title': 'Type'},
                   ) #plot_bgcolor='rgba(0,0,0,0) to remove background color

go.Figure(data1, layout1).update_yaxes(categoryorder='total ascending')

- Average market value per punk type \
  Pricing premium are given to rare punk types due to its scarcity

In [26]:
#remove extreme value from latest market value 
df2_noex = df2[(df2['latest_market_value'] < 10000000) & (df2['latest_market_value'] > 0.01)]
df2_noex

,number,punk_type,initial_owner,current_owner,transaction_type,transaction_frequency,initial_market_value,latest_market_value,initial_TXD,latest_TXD,initial_TXY,latest_TXY
0,0,Female,0xc352b5,0xe08c32,Bid Withdrawn,34,219.0,1030000.0,2017-06-23,2021-05-10,2017,2021
3,3,Male,0xc352b5,0xc352b5,Bid Withdrawn,14,13.0,11638.0,2017-06-26,2020-10-04,2017,2020
4,4,Male,0xc352b5,0xc352b5,Bid Withdrawn,11,28.0,109026.0,2017-06-25,2021-05-13,2017,2021
5,5,Female,0xc352b5,WrappedC?€?,Bid,13,15.0,11322.0,2017-06-26,2021-04-29,2017,2021
6,6,Male,0xc352b5,0xc352b5,Bid Withdrawn,6,15.0,10126.0,2017-06-26,2021-01-04,2017,2021
...,...,...,...,...,...,...,...,...,...,...,...,...
9994,9994,Female,sov,0x69021a,Bid,2,0.0,527.0,2021-04-25,2021-04-27,2021,2021
9995,9995,Female,Punkofth?€?,0x758353,Bid Withdrawn,6,16.0,63960.0,2017-06-28,2021-04-20,2017,2021
9996,9996,Male,0xa9bfa9,0xa9bfa9,Bid Withdrawn,4,11.0,1338.0,2017-08-04,2021-01-20,2017,2021
9997,9997,Zombie,TJ2010,0x279679,Offered,36,46.0,1480000.0,2017-06-24,2021-05-17,2017,2021


In [27]:
punk_types_avg = df2_noex.groupby('punk_type')['latest_market_value'].agg('mean').round()

data2 = go.Bar(x=punk_types_avg .values, y=punk_types_avg.index, orientation='h', text=punk_types_avg .values, textposition='auto')

layout2 = go.Layout(title='Average market value per punk type',
                    xaxis={'title': 'Average market value $USD'},
                    yaxis={'title': 'Type'},
                   ) 

go.Figure(data2, layout2).update_yaxes(categoryorder='total ascending')

- Pricing distribution of punk type\
  After price of rare punk types was pumped up, people tend to dig gold in common punk types (male and female)

In [28]:
#remove extreme value from latest market value and set transaction type as bid
df2_noex = df2[(df2['latest_market_value']<10000000) & (df2['latest_market_value']>0.01) & (df2['transaction_type'] == 'Bid')]

data3 = go.Box(x=df2_noex['punk_type'],
               y=df2_noex['latest_market_value'],
               boxpoints='all')

layout3 = go.Layout(title='Bid - market value distribution of punk type',
                    xaxis={'title':'Type'},
                    yaxis={'title':'Average market value USD$'},
                   ) #plot_bgcolor='rgba(0,0,0,0) to remove background color

go.Figure(data3, layout3)            

- Quantity distribution of punk attributes\
  Punks really love earrings, smokings and colorful lipsticks\
  Tiara and Beanie are far away from punk style

In [29]:
punk_attributes = df1['attribute'].value_counts().to_frame()

punk_attributes

,attribute
Earring,1900
Cigarette,739
Hot Lipstick,547
Purple Lipstick,492
Mole,487
...,...
Orange Side,50
Pilot Helmet,40
Choker,37
Tiara,37


- Punk attributes value map\
  Price premium is also allocated to rare attribute

In [30]:
#Scatter chart
punk_att = df[(df['latest_market_value']<10000000)].groupby('attribute')\
                                             .agg({'latest_market_value':['mean', 'count']})

data4 = go.Scatter(y=punk_att['latest_market_value']['mean'],
                   x=punk_att['latest_market_value']['count'],
                   mode='markers',
                   text=punk_att.index)
                   
layout4 = go.Layout(title='Attribute value proposition',
                    xaxis={'title':'Quantity of attribute'},
                    yaxis={'title':'Average market value USD$'})

go.Figure(data4,layout4)

- Crypto punk ownership\
  Initially, 61% of total crypto punks were controlled by 15 owners. After 4 years, top 15 owners only have 28% of total crypto   punks. Owners capture crypto punk as short-term investment rather than holding it long time.

In [31]:
punk_own_initial = df2.groupby('initial_owner').agg({'number': 'count'})\
                                               .sort_values(by='number', ascending=False)\
                                               .rename(columns={'number': 'Punk Quantity'}) 

punk_own_initial['percent %'] = punk_own_initial['Punk Quantity'] / 10000 * 100

punk_own_initial['cum_percent %'] = punk_own_initial['Punk Quantity'].cumsum() / 10000 * 100

punk_own_initial.head(15)

,Punk Quantity,percent %,cum_percent %
initial_owner,,,
0xc352b5,1020,10.20,10.20
0x00d7c9,1005,10.05,20.25
Hemba,919,9.19,29.44
0xcbd482,731,7.31,36.75
0x00bd9f,504,5.04,41.79
0x6f4a2d,319,3.19,44.98
0x6611fe,289,2.89,47.87
0xcc7c33,200,2.00,49.87
0xe45df5,195,1.95,51.82


In [32]:
punk_own_current = df2.groupby('current_owner').agg({'number': 'count'})\
                                               .sort_values(by='number', ascending=False)\
                                               .rename(columns={'number': 'Punk Quantity'}) 

punk_own_current['percent %'] = punk_own_current['Punk Quantity'] / 10000 * 100

punk_own_current['cum_percent %'] = punk_own_current['Punk Quantity'].cumsum() / 10000 * 100

punk_own_current.head(15)

,Punk Quantity,percent %,cum_percent %
current_owner,,,
0xc352b5,430,4.30,4.30
wilcox.e?€?,281,2.81,7.11
WrappedC?€?,276,2.76,9.87
0x577ebc,269,2.69,12.56
0xb88f61,215,2.15,14.71
0x6611fe,168,1.68,16.39
0x6f4a2d,162,1.62,18.01
0xcc7c33,156,1.56,19.57
0x69021a,147,1.47,21.04


- Crypto punk transaction frequency\
  10% of crypto punk owners have no intention to trade even after 4 years, they treat the pixel image as art for their
  collections. \
  Total trading frequency is relatively low due to character of NFT.


In [33]:
data5 = go.Histogram(x=df2['transaction_frequency'])

layout5 = go.Layout(title='2017-2021 Transaction frequency ',
                    xaxis={'title':'Trading frequency'},
                    yaxis={'title':'Total number'})

go.Figure(data5, layout5)

